use loc[i, field] instead of iloc[i, field], iloc is very slow.

In [1]:
import sys
sys.path = ['/root/autodl-tmp/xiaolong/WorkSpace/Amazon-KDDCUP-23/RecStudio/'] + sys.path
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
import random
import numpy as np
import pandas as pd
import cudf, itertools
import scipy.sparse as ssp
from functools import lru_cache, partial
from tqdm import tqdm, trange
from collections import Counter, defaultdict
import torch
import pickle

/root/autodl-tmp/xiaolong/.conda/envs/torch12/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def cast_dtype(df : pd.DataFrame):
    for k in df.columns:
        dt = type(df[k].iloc[0])
        if 'float' in str(dt):
            df[k] = df[k].astype('float32')
        elif 'int' in str(dt):
            df[k] = df[k].astype('int32')
        elif dt == list:
            dt_ = type(df.iloc[0][k][0])
            if 'float' in str(dt_):
                df[k] = df[k].apply(lambda x : np.array(x, dtype=np.float32))
            elif 'int' in str(dt_):
                df[k] = df[k].apply(lambda x : np.array(x, dtype=np.int32))

In [3]:
def _load_cache(path):
    with open(path, 'rb') as f:
        download_obj = pickle.load(f)
    return download_obj

In [26]:
def get_scores(merged_candidates_df, query_embeddings, product_embeddings):
    batch_size = 2048
    num_iter = (len(merged_candidates_df) - 1) // batch_size + 1
    score_list = []
    with torch.no_grad():
        for i in tqdm(range(num_iter)):
            st, ed = i * batch_size, (i + 1) * batch_size 
            batch_sess = merged_candidates_df.iloc[st : ed]
            batch_sess_id = torch.tensor(batch_sess['sess_id'].tolist(), dtype=torch.long, device=query_embeddings.device)
            batch_product_id = torch.tensor(batch_sess['dataset_id'].tolist(), dtype=torch.long, device=product_embeddings.device)
            query_emb = query_embeddings[batch_sess_id]
            product_emb = product_embeddings[batch_product_id]
            batch_score = (query_emb * product_emb).sum(dim=-1) 
            score_list.append(batch_score.cpu())
        score_list = torch.cat(score_list, dim=0).cpu().tolist()
        return score_list 

In [5]:
def normalize_scores(score_df, score_name, normalized_score_name):
    # score_df_g = cudf.from_pandas(score_df)
    score_df['exp_score'] = np.exp(score_df[score_name].to_numpy())
    scores_sum = score_df[['sess_id', 'exp_score']].groupby('sess_id').sum()
    scores_sum.reset_index(inplace=True)
    scores_sum = scores_sum.sort_values(by=['sess_id'], ascending=True)
    scores_sum.reset_index(drop=True, inplace=True)
    scores_sum.rename(columns={'exp_score' : 'score_sum'}, inplace=True)

    merged_score_df = score_df.merge(scores_sum, how='left', left_on=['sess_id'], right_on=['sess_id'])
    merged_score_df = merged_score_df.sort_values(by=['sess_id', 'product'])
    merged_score_df.reset_index(drop=True, inplace=True)
    
    # merged_score_df = merged_score_df_g.to_pandas(merged_score_df_g)
    score_df[normalized_score_name] = merged_score_df['exp_score'] / merged_score_df['score_sum']
    score_df['exp_score'] = merged_score_df['exp_score']
    score_df['score_sum'] = merged_score_df['score_sum']

    # del scores_sum_g
    # del merged_score_df_g 

# Merge test score

In [45]:
FIELD_NAME = 'sasrec_scores_2'

In [46]:
merged_candidates_feature_test_path = '/root/autodl-tmp/xiaolong/WorkSpace/Amazon-KDDCUP-23/XGBoost/candidates/merged_candidates_test_feature.parquet'
test_sessions_path = '/root/autodl-tmp/xiaolong/WorkSpace/Amazon-KDDCUP-23/raw_data/sessions_test_task1.csv'

In [47]:
DE_product_embeddings_path = '/root/autodl-tmp/xiaolong/WorkSpace/Amazon-KDDCUP-23/candidates/query_embeddings/SASRec_Next/kdd_cup_2023_DE/product_embeddings_2023-05-24-15-30-10.pt'
DE_test_embeddings_path = '/root/autodl-tmp/xiaolong/WorkSpace/Amazon-KDDCUP-23/candidates/query_embeddings/SASRec_Next/kdd_cup_2023_DE/predict_embeddings_2023-05-24-15-30-48.pt'
JP_product_embeddings_path = '/root/autodl-tmp/xiaolong/WorkSpace/Amazon-KDDCUP-23/candidates/query_embeddings/SASRec_Next/kdd_cup_2023_JP/product_embeddings_2023-05-24-15-31-02.pt'
JP_test_embeddings_path = '/root/autodl-tmp/xiaolong/WorkSpace/Amazon-KDDCUP-23/candidates/query_embeddings/SASRec_Next/kdd_cup_2023_JP/predict_embeddings_2023-05-24-15-31-42.pt'
UK_product_embeddings_path = '/root/autodl-tmp/xiaolong/WorkSpace/Amazon-KDDCUP-23/candidates/query_embeddings/SASRec_Next/kdd_cup_2023_UK/product_embeddings_2023-05-24-15-31-54.pt'
UK_test_embeddings_path = '/root/autodl-tmp/xiaolong/WorkSpace/Amazon-KDDCUP-23/candidates/query_embeddings/SASRec_Next/kdd_cup_2023_UK/predict_embeddings_2023-05-24-15-32-33.pt'

In [48]:
@lru_cache(maxsize=1)
def read_merged_candidates_feature_test():
    return pd.read_parquet(merged_candidates_feature_test_path, engine='pyarrow')

@lru_cache(maxsize=1)
def read_test_sessions():
    return pd.read_csv(test_sessions_path)

In [49]:
merged_candidates = read_merged_candidates_feature_test()
test_sessions = read_test_sessions()
EMBED_DIM = 128
merged_candidates.sort_values(by=['sess_id', 'product'], inplace=True)
merged_candidates.reset_index(drop=True, inplace=True)

In [50]:
# sess embeddings 
test_DE_query_emb = torch.load(DE_test_embeddings_path, map_location='cpu')
test_JP_query_emb = torch.load(JP_test_embeddings_path, map_location='cpu')
test_UK_query_emb = torch.load(UK_test_embeddings_path, map_location='cpu')
test_query_embeddings = torch.cat(
    [test_DE_query_emb[test_sessions['locale'] == 'DE'], test_JP_query_emb[test_sessions['locale'] == 'JP'], test_UK_query_emb[test_sessions['locale'] == 'UK']],
    dim=0)

In [51]:
# product embeddings 
DE_product_emb = torch.load(DE_product_embeddings_path, map_location='cpu')
JP_product_emb = torch.load(JP_product_embeddings_path, map_location='cpu')
UK_product_emb = torch.load(UK_product_embeddings_path, map_location='cpu')
product_embeddings = torch.cat([DE_product_emb, JP_product_emb, UK_product_emb], dim=0)

In [52]:
DE_dataset_cache = '/root/autodl-tmp/xiaolong/WorkSpace/Amazon-KDDCUP-23/.recstudio/cache/c76eddf0a07106ffcce7ce8010856a3b'
JP_dataset_cache = '/root/autodl-tmp/xiaolong/WorkSpace/Amazon-KDDCUP-23/.recstudio/cache/81a71d0a18766af84b3beab69bf53e69'
UK_dataset_cache = '/root/autodl-tmp/xiaolong/WorkSpace/Amazon-KDDCUP-23/.recstudio/cache/250dbc09c30162452e00486051e47756'
DE_train_dataset, DE_valid_dataset = _load_cache(DE_dataset_cache)
JP_train_dataset, JP_valid_dataset = _load_cache(JP_dataset_cache)
UK_train_dataset, UK_valid_dataset = _load_cache(UK_dataset_cache)
locale_map = {
    'DE' : DE_train_dataset.field2token2idx['product_id'], 
    'JP' : JP_train_dataset.field2token2idx['product_id'], 
    'UK' : UK_train_dataset.field2token2idx['product_id']
    }

In [53]:
DE_product_list, DE_id_list = list(zip(*locale_map['DE'].items()))
JP_product_list, JP_id_list = list(zip(*locale_map['JP'].items()))
UK_product_list, UK_id_list = list(zip(*locale_map['UK'].items()))
product_list = list(DE_product_list) + list(JP_product_list) + list(UK_product_list)
id_list = list(DE_id_list) + list(JP_id_list) + list(UK_id_list)
locale_list = ['DE'] * len(DE_id_list) + ['JP'] * len(JP_id_list) + ['UK'] * len(UK_id_list)
product_id_df = pd.DataFrame({'locale' : locale_list, 'product' : product_list, 'dataset_id' : id_list})

In [54]:
merged_candidates_sasrec = merged_candidates[['sess_id', 'sess_locale', 'product']].copy()

In [17]:
# merged_candidates_sasrec_g = cudf.from_pandas(merged_candidates_sasrec)
# product_id_df_g = cudf.from_pandas(product_id_df)

In [18]:
# merged_candidates_sasrec_score_g = merged_candidates_sasrec_g.merge(product_id_df_g, how='left', left_on=['sess_locale', 'product'], right_on=['locale', 'product'])
# merged_candidates_sasrec_score_g['dataset_id'] = merged_candidates_sasrec_score_g['dataset_id'].fillna(0)
# merged_candidates_sasrec_score_g.drop(columns=['locale'], inplace=True)
# merged_candidates_sasrec_score_g = merged_candidates_sasrec_score_g.sort_values(by=['sess_id', 'product'])
# merged_candidates_sasrec_score_g.reset_index(drop=True, inplace=True)
# merged_candidates_sasrec_score = merged_candidates_sasrec_score_g.to_pandas()

In [55]:
merged_candidates_sasrec_score = merged_candidates_sasrec.merge(product_id_df, how='left', left_on=['sess_locale', 'product'], right_on=['locale', 'product'])
merged_candidates_sasrec_score['dataset_id'] = merged_candidates_sasrec_score['dataset_id'].fillna(0)
merged_candidates_sasrec_score.drop(columns=['locale'], inplace=True)
merged_candidates_sasrec_score = merged_candidates_sasrec_score.sort_values(by=['sess_id', 'product'])
merged_candidates_sasrec_score.reset_index(drop=True, inplace=True)

In [19]:
# del merged_candidates_sasrec_score_g
# del product_id_df_g
# del merged_candidates_sasrec_g

In [56]:
locale_offset = {'DE' : 0, 'JP' : len(DE_product_list), 'UK' : len(DE_product_list) + len(JP_product_list)}
for locale in ['DE', 'JP', 'UK']:
    merged_candidates_sasrec_score['dataset_id'][merged_candidates_sasrec_score['sess_locale'] == locale] = \
        merged_candidates_sasrec_score['dataset_id'][merged_candidates_sasrec_score['sess_locale'] == locale] + locale_offset[locale]

/tmp/ipykernel_317463/2146751149.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged_candidates_sasrec_score['dataset_id'][merged_candidates_sasrec_score['sess_locale'] == locale] = \


In [57]:
merged_candidates_sasrec_score[FIELD_NAME] = get_scores(merged_candidates_sasrec_score, test_query_embeddings, product_embeddings)

  0%|          | 0/32441 [00:00<?, ?it/s]/tmp/ipykernel_317463/150055658.py:10: DeprecationWarning: an integer is required (got type float).  Implicit conversion to integers using __int__ is deprecated, and may be removed in a future version of Python.
  batch_product_id = torch.tensor(batch_sess['dataset_id'].tolist(), dtype=torch.long, device=product_embeddings.device)
100%|██████████| 32441/32441 [02:47<00:00, 193.79it/s]


In [58]:
normalize_scores(merged_candidates_sasrec_score, FIELD_NAME, 'normalized_'+FIELD_NAME)

In [59]:
merged_candidates[FIELD_NAME] = merged_candidates_sasrec_score[FIELD_NAME]
merged_candidates['normalized_'+FIELD_NAME] = merged_candidates_sasrec_score['normalized_'+FIELD_NAME]

In [56]:
cast_dtype(merged_candidates)
merged_candidates.to_parquet(merged_candidates_feature_test_path, engine='pyarrow')

In [60]:
merged_candidates

,sess_id,sess_locale,product,sess_avg_price,product_price,product_freq,sasrec_scores_2,normalized_sasrec_scores_2
0,0,DE,B000JG2RAG,25.195269,23.190001,24.0,7.665308,6.342882e-06
1,0,DE,B000RYSOUW,25.195269,6.900000,5.0,-2.951060,1.554736e-10
2,0,DE,B000UGZVQM,25.195269,21.990000,4.0,3.977920,1.588087e-07
3,0,DE,B000Z6JN7K,25.195269,13.170000,7.0,6.600063,2.186032e-06
4,0,DE,B003CYK6FU,25.195269,11.990000,4.0,3.741957,1.254287e-07
...,...,...,...,...,...,...,...,...
66438766,316970,UK,B0BJJMGPJ7,16.950001,7.990000,27.0,8.033210,3.860075e-06
66438767,316970,UK,B0BJTQQWLG,16.950001,9.880000,6.0,5.604142,3.401458e-07
66438768,316970,UK,B0BJV3RL4H,16.950001,22.097065,7.0,9.146974,1.175702e-05
66438769,316970,UK,B0BK7SPC84,16.950001,5.960000,0.0,-10.383047,3.877189e-14


In [61]:
merged_candidates[(merged_candidates['sess_id'] == 200000)].sort_values(by='normalized_sasrec_scores_2', ascending=False)[['sess_locale', 'product', 'normalized_sasrec_scores_2', 'sasrec_scores_2']][:20]

,sess_locale,product,normalized_sasrec_scores_2,sasrec_scores_2
41680103,JP,B07WVXGFCJ,0.210410,19.104816
41680231,JP,B09Y32HQS5,0.122188,18.561321
41680293,JP,B0BFQGJRBY,0.104701,18.406866
41680203,JP,B09NXS2J43,0.079784,18.135077
41680216,JP,B09TVFSQ7F,0.066238,17.949009
41680277,JP,B0BBH5S35X,0.037233,17.372955
41680089,JP,B07H27J698,0.035418,17.322983
41680237,JP,B0B24Z6MCB,0.033055,17.253918
41680235,JP,B09Z2KGR1Y,0.031099,17.192924
41680101,JP,B07SDHFX36,0.030940,17.187799
